In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import math
import nltk
nltk.download('all')
from textblob import TextBlob as tb

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

In [3]:
import nltk
from nltk import data
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import re, collections
from sklearn.feature_extraction.text import CountVectorizer

# Tokenize a sentence into words
def sentence_to_wordlist(raw_sentence):

    clean_sentence = re.sub("[^a-zA-Z0-9]"," ", raw_sentence)
    tokens = nltk.word_tokenize(clean_sentence)    
    return tokens


# tokenizing an essay into a list of word lists
def tokenize(essay):
    stripped_essay = essay.strip()
    
    tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sentences = tokenizer.tokenize(stripped_essay)
    
    tokenized_sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            tokenized_sentences.append(sentence_to_wordlist(raw_sentence))
    
    return tokenized_sentences


# calculating average word length in an essay
def avg_word_len(essay):    
    clean_essay = re.sub(r'\W', ' ', essay)
    words = nltk.word_tokenize(clean_essay)
    
    return sum(len(word) for word in words) / len(words)


# calculating number of words in an essay
def word_count(essay):    
    clean_essay = re.sub(r'\W', ' ', essay)
    words = nltk.word_tokenize(clean_essay)
    
    return len(words)


# calculating number of characters in an essay
def char_count(essay):    
    clean_essay = re.sub(r'\s', '', str(essay).lower())
    
    return len(clean_essay)


# calculating number of sentences in an essay
def sent_count(essay):    
    sentences = nltk.sent_tokenize(essay)
    
    return len(sentences)

# calculating number of lemmas per essay
def count_lemmas(essay):    
    tokenized_sentences = tokenize(essay)      
    
    lemmas = []
    wordnet_lemmatizer = WordNetLemmatizer()
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence) 
        
        for token_tuple in tagged_tokens:
        
            pos_tag = token_tuple[1]
        
            if pos_tag.startswith('N'): 
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('J'):
                pos = wordnet.ADJ
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('V'):
                pos = wordnet.VERB
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            elif pos_tag.startswith('R'):
                pos = wordnet.ADV
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
            else:
                pos = wordnet.NOUN
                lemmas.append(wordnet_lemmatizer.lemmatize(token_tuple[0], pos))
    
    lemma_count = len(set(lemmas))
    
    return lemma_count


# checking number of misspelled words
def count_spell_error(essay):    
    clean_essay = re.sub(r'\W', ' ', str(essay).lower())
    clean_essay = re.sub(r'[0-9]', '', clean_essay)
    
    #big.txt: It is a concatenation of public domain book excerpts from Project Gutenberg 
    #         and lists of most frequent words from Wiktionary and the British National Corpus.
    #         It contains about a million words.
    data = open('big.txt').read()    
    words_ = re.findall('[a-z]+', data.lower())    
    word_dict = collections.defaultdict(lambda: 0)
                       
    for word in words_:
        word_dict[word] += 1
                       
    clean_essay = re.sub(r'\W', ' ', str(essay).lower())
    clean_essay = re.sub(r'[0-9]', '', clean_essay)
                        
    mispell_count = 0
    
    words = clean_essay.split()
                        
    for word in words:
        if not word in word_dict:
            mispell_count += 1
    
    return mispell_count


# calculating number of nouns, adjectives, verbs and adverbs in an essay
def count_pos(essay):    
    tokenized_sentences = tokenize(essay)
    
    noun_count = 0
    adj_count = 0
    verb_count = 0
    adv_count = 0
    
    for sentence in tokenized_sentences:
        tagged_tokens = nltk.pos_tag(sentence)
        
        for token_tuple in tagged_tokens:
            pos_tag = token_tuple[1]
        
            if pos_tag.startswith('N'): 
                noun_count += 1
            elif pos_tag.startswith('J'):
                adj_count += 1
            elif pos_tag.startswith('V'):
                verb_count += 1
            elif pos_tag.startswith('R'):
                adv_count += 1
            
    return noun_count, adj_count, verb_count, adv_count
    
# getiing Bag of Words (BOW) counts
def get_count_vectors(essays):
    
    vectorizer = CountVectorizer(max_features = 10000, ngram_range=(1, 3), stop_words='english')        #BOW
    
    count_vectors = vectorizer.fit_transform(essays)             #lear vocunulary dictionary and return document matrix
    
    feature_names = vectorizer.get_feature_names()                    #TFIDF
    
    return feature_names, count_vectors

# extracting essay features
def extract_features(data):
    
    features = data.copy()
    
    features['char_count'] = features['essay'].apply(char_count)
    
    features['word_count'] = features['essay'].apply(word_count)
    
    features['sent_count'] = features['essay'].apply(sent_count)
    
    features['avg_word_len'] = features['essay'].apply(avg_word_len)
    
    features['lemma_count'] = features['essay'].apply(count_lemmas)
    
    features['spell_err_count'] = features['essay'].apply(count_spell_error)
    
    features['noun_count'], features['adj_count'], features['verb_count'], features['adv_count'] = zip(*features['essay'].map(count_pos))
    
    return features

In [4]:
%cd /content/drive/MyDrive/FullProject

/content/drive/MyDrive/FullProject


In [5]:
pip install flask_ngrok

In [6]:
pip install pandas

In [7]:
pip install simpletransformers

     |████████████████████████████████| 225kB 8.0MB/s 
     |████████████████████████████████| 1.2MB 14.0MB/s 
     |████████████████████████████████| 2.5MB 36.4MB/s 
     |████████████████████████████████| 133kB 39.8MB/s 
     |████████████████████████████████| 1.8MB 44.5MB/s 
     |████████████████████████████████| 7.8MB 34.1MB/s 
     |████████████████████████████████| 245kB 40.1MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 3.3MB 26.8MB/s 
     |████████████████████████████████| 81kB 8.0MB/s 
     |████████████████████████████████| 901kB 30.9MB/s 
     |████████████████████████████████| 174kB 48.1MB/s 
     |████████████████████████████████| 133kB 35.6MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 
     |████████████████████████████████| 81kB 8.3MB/s 
     |████████████████████████████████| 4.2MB 27.7MB/s 
     |████████████████████████████████| 112kB 39.0MB/s 
     |████████████████████████████████| 245kB 43.0MB/s 

In [ ]:
from __future__ import division, print_function
import numpy as np

# !pip install simpletransformers
from simpletransformers.t5 import T5Model
# Flask utils
from flask import Flask,request, render_template
from flask_ngrok import run_with_ngrok
#ts utils
import numpy as np
import pickle
import pandas as pd


pkl_filename = "/content/drive/MyDrive/FullProject/testmodel.pkl"
with open(pkl_filename, 'rb') as file:
    pickle_model = pickle.load(file)

# Define a flask app
app = Flask(__name__)
run_with_ngrok(app)

@app.route('/', methods=['GET'])
def index():
    # Main page
    print("index")
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        try:
            user_input= request.form['input_text']
            pred_params = {
                    'max_seq_length': 512,
                    'use_multiprocessed_decoding': False
                    }
            # list of sentences
            input_list=[]

            input_text=user_input.split('.') #list of sentences
            print
            str="" #para
            count=0 #length pf para
            for sentence in input_text:
                str=str+sentence
                count=count+len(sentence)
                if(count>128):
                    input_list.append(str)
                    count=0
                    str=""
            

            mo = T5Model('t5', 'outputs/best_model', args=pred_params, use_cuda=False)
            pred = mo.predict(input_list)
            predicted_output=''
            for i in range(len(pred)):
                predicted_output=predicted_output+pred[i]+"."
            print("predicted output is : ",predicted_output)
            if(len(user_input.split(' '))>300):
                disabled=False
            else:
                disabled=True
            return render_template('prediction.html',original_text=user_input,summarized_text=predicted_output,disabled=disabled)
        except Exception as e:
            print(e)
            return render_template('index.html')
    return None

@app.route('/grading', methods=['GET', 'POST'])
def grade():
    if request.method == 'POST':
        try:
            user_input= request.form['input_text']
            print(user_input)

            inp='''

            My first campaign as spokesman and strategist for Tony Blair was in 1997, three years in the planning after he had become leader of the Opposition  Labour Party. Some of the principles of strategy we applied back then would certainly apply to a modern day election. But their tactical execution almost certainly would not. Politicians and their strategists have to adapt to change as well as lead it. Seguela gives some interesting insights into those who have adapted well, and those who have done less well. He clearly adores former President Lula of Brazil and you can feel his yearning for a French leader who can somehow combine hard-headed strategy with human empathy in the same way as a man who left office with satisfaction ratings of 87percent. Seguela probably remains best known in political circles for his role advising Francois Mitterrand. Yet wheras I am ‘tribal Labour’, and could not imagine supporting a Conservative Party candidate in the UK, Seguela came out as a major supporter of Nicolas Sarkozy. I wonder if one of the reasons was not a frustration that large parts of the left in France remain eternally suspicious of modern communications techniques and styles which, frankly, no modern leader in a modern democracy can ignore. How he or she adapts to, or uses, them is up to them. But you cannot stand aside and imagine the world has not changed.

            '''
            # creating a dataframe with essay
            df=pd.DataFrame({'essay':[user_input],'essay_set':[1]})

            #  Vectorizing array 
            a,b= get_count_vectors(df['essay'])

            # matrix to numpy array
            xcv = b.toarray()

            # As model is trained with shape(600*10000) setting extra characters to zero
            new=np.pad(xcv[0], (0, (10000-len(xcv[0]))), 'constant')

            # Extracting features for the given essay for different metrics
            fs1 = extract_features(df)

            # Adding vectors and features
            x = np.concatenate((fs1.iloc[:, 1:].to_numpy(), [new]), axis = 1)
            # predicting the output using the saved model 
            predicted_grade= pickle_model.predict(x)

            predicted_grade=np.around(predicted_grade)
            features=fs1.drop(columns=['essay','essay_set'])
            features_list=[features.columns.values.tolist()] + features.values.tolist()
            print(features_list)
            f_list={}
            for i in range(len(features_list[0])):
                f_list[features_list[0][i]]=features_list[1][i]
            print(f_list)
            
            
            
            return render_template('grading.html',original_text=user_input,predicted_grade=min(predicted_grade[0],10),features=f_list)
        except Exception as e:
            print(e)
            return render_template('index.html')
    return None
if __name__ == '__main__':
  app.run()




 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Lasso from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator GridSearchCV from version 0.24.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://95ff51a1664d.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
index


127.0.0.1 - - [03/Jul/2021 02:46:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2021 02:46:03] "GET /static/js/main.js HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2021 02:46:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jul/2021 02:46:03] "GET /static/css/main.css HTTP/1.1" 200 -


index


...

predicted output is :  Mercury is the smallest planet in our solar system: Earth.Mercury can go up to 450 degrees Celsius during day time.Mercury has a diameter of 4,878 km, no satellite like Earth Venus.Venus is the brightest planet in the world.Astronomers see traces of internal geological activity on Venus.Earth is only planet in our solar system where life is possible.Earth has a diameter of 12,760 km, Earth has one satellite that is the moon.Mars is the fourth planet from Sun.Mars planet is a cold planet, it has geological features similar to Earth.Mars has traces of frozen ice caps and it has a diameter of 6,787 km.Jupiter is the largest planet in our solar system.Jupiter has 139,822 km, 79 satellites over Earth and Mars.Saturn is sixth planet from Sun.Saturn's atmosphere is similar to Jupiter because it is composed of hydrogen.Jupiter has less planet from Sun: Astronomy.Uranus planet has blue tint Uranus core colder than giants.Uranus has a diameter of 51,120 km.Neptune is the s

127.0.0.1 - - [03/Jul/2021 03:37:36] "POST /grading HTTP/1.1" 200 -


[['char_count', 'word_count', 'sent_count', 'avg_word_len', 'lemma_count', 'spell_err_count', 'noun_count', 'adj_count', 'verb_count', 'adv_count'], [2748.0, 625.0, 45.0, 4.2976, 216.0, 28.0, 188.0, 73.0, 96.0, 28.0]]
{'char_count': 2748.0, 'word_count': 625.0, 'sent_count': 45.0, 'avg_word_len': 4.2976, 'lemma_count': 216.0, 'spell_err_count': 28.0, 'noun_count': 188.0, 'adj_count': 73.0, 'verb_count': 96.0, 'adv_count': 28.0}


127.0.0.1 - - [03/Jul/2021 03:53:07] "GET / HTTP/1.1" 200 -


index
